# Task Library

In [ ]:
# | default_exp asana.task

In [ ]:
# | exporti

from __future__ import annotations

from dataclasses import dataclass, field
from typing import List, Optional

import datetime as dt

from mdutils.mdutils import MdUtils

from nbdev.showdoc import patch_to

import domolibrary_extensions.utils as ut
import domolibrary_extensions.client as gd
import domolibrary_extensions.asana.auth as aa
import domolibrary_extensions.asana.user as au
import domolibrary_extensions.asana.project as ap

In [ ]:
# | hide
from nbdev.showdoc import show_doc

from dotenv import load_dotenv
import os
import json

load_dotenv("../.env")

True

In [ ]:
# | export


@dataclass
class AsanaSection:
    id: str
    name: str
    resource_type: str
    auth: aa.AsanaAuth = field(repr=False)

    @classmethod
    def _from_json(cls, data: dict, auth=auth) -> "AsanaSection":
        return cls(
            id=data.get("gid"),
            name=data.get("name"),
            resource_type=data.get("resource_type"),
            auth=auth,
        )


@dataclass
class AsanaMembership:
    """relates a section to a project"""

    project: ap.AsanaProject
    section: AsanaSection
    auth: aa.AsanaAuth = field(repr=False)

    @classmethod
    def _from_json(cls, data: dict, auth: aa.AsanaAuth) -> "AsanaMembership":
        project = (
            ap.AsanaProject._from_json(data["project"], auth=auth)
            if data.get("project")
            else None
        )
        section = (
            AsanaSection._from_json(data["section"], auth=auth)
            if data.get("section")
            else None
        )
        return cls(project=project, section=section, auth=auth)

    def to_text(self):
        return f"{self.project.name} -> {self.section.name}"


@dataclass
class AsanaStory:
    id: str
    created_at: dt.DateTime
    created_by: au.AsanaUser
    text: str
    type: str
    resource_subtype: str

    auth: aa.AsanaAuth = field(repr=False)

    @classmethod
    def _from_json(cls, data: dict, auth: aa.AsanaAuth) -> AsanaStory:
        created_by = (
            au.AsanaUser._from_json(data["created_by"], auth=auth)
            if data.get("created_by")
            else None
        )

        return cls(
            id=data.get("gid"),
            created_at=ut.convert_str_to_date(data.get("created_at")),
            created_by=created_by,
            text=data.get("text"),
            type=data.get("type"),
            resource_subtype=data.get("resource_subtype"),
            auth=auth,
        )

    def to_text(self):
        return f"{self.created_at.date()} - {self.created_by.name} - {self.text}"

# Asana Task

In [ ]:
# | export
@dataclass
class AsanaTask:
    id: str
    name: str
    workspace_id: str

    auth: aa.AsanaAuth = field(repr=False)
    assignee: au.AsanaUser = None

    assignee_status: str = None

    is_completed: bool = None

    created_at: dt.datetime = None
    completed_on: dt.datetime = None
    due_on: dt.datetime = None
    modified_at: dt.datetime = None

    memberships: List[dict] = None

    notes: str = None

    parent: Optional[dict] = None
    permalink_url: str = None
    projects: List[ap.AsanaProject] = None
    stories: List[AsanaStory] = None

    tags: List[dict] = None

    @classmethod
    def _from_json(cls, obj: dict, auth: aa.AsanaAuth) -> AsanaTask:
        assignee = (
            au.AsanaUser._from_json(obj.get("assignee"), auth=auth)
            if obj.get("assignee")
            else None
        )

        projects = [
            ap.AsanaProject._from_json(project, auth=auth)
            for project in obj.get("projects", [])
        ]

        memberships = [
            AsanaMembership._from_json(member_obj, auth=auth)
            for member_obj in obj.get("memberships", [])
        ]
        return cls(
            id=obj.get("gid"),
            name=obj.get("name"),
            auth=auth,
            workspace_id=obj.get("workspace", {}).get("gid"),
            assignee=assignee,
            is_completed=obj.get("completed"),
            assignee_status=obj.get("assignee_status"),
            completed_on=ut.convert_str_to_date(obj.get("completed_at")),
            created_at=ut.convert_str_to_date(obj.get("created_at")),
            due_on=ut.convert_str_to_date(obj.get("due_on")),
            modified_at=ut.convert_str_to_date(obj.get("modified_at")),
            memberships=memberships,
            notes=obj.get("notes"),
            parent=obj.get("parent"),
            permalink_url=obj.get("permalink_url"),
            tags=obj.get("tags", []),
            projects=projects,
        )

In [ ]:
show_doc(AsanaTask)

---

[source](https://github.com/jaewilson07/domolibrary_extensions/blob/main/domolibrary_extensions/asana/task.py#L103){target="_blank" style="float:right; font-size:smaller"}

### AsanaTask

>      AsanaTask (id:str, name:str, workspace_id:str,
>                 auth:domolibrary_extensions.asana.auth.AsanaAuth,
>                 assignee:domolibrary_extensions.asana.user.AsanaUser=None,
>                 assignee_status:str=None, is_completed:bool=None,
>                 created_at:datetime.datetime=None,
>                 completed_on:datetime.datetime=None,
>                 due_on:datetime.datetime=None,
>                 modified_at:datetime.datetime=None,
>                 memberships:List[dict]=None, notes:str=None,
>                 parent:Optional[dict]=None, permalink_url:str=None,
>                 projects:List[domolibrary_extensions.asana.project.AsanaProject]=None,
>                 stories:List[__main__.AsanaStory]=None, tags:List[dict]=None)

In [ ]:
# | exporti
@patch_to(AsanaTask, cls_method=True)
async def get_by_id(
    cls, auth, task_id, debug_api: bool = False, return_raw: bool = False
):
    url = f"{auth.base_url}/tasks/{task_id}"

    res = await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    return cls._from_json(res.response["data"], auth=auth)

In [ ]:
show_doc(AsanaTask.get_by_id)

---

[source](https://github.com/jaewilson07/domolibrary_extensions/blob/main/domolibrary_extensions/asana/task.py#L170){target="_blank" style="float:right; font-size:smaller"}

### AsanaTask.get_by_id

>      AsanaTask.get_by_id (auth, task_id, debug_api:bool=False,
>                           return_raw:bool=False)

In [ ]:
asana_pat = json.loads(os.environ["ASANA_PUBLIC"])["ASANA_PAT"]
workspace_id = "1206043185525448"
asana_auth = aa.AsanaAuth(token=asana_pat, workspace_id=workspace_id)

task_id = "1206043185525463"
await AsanaTask.get_by_id(task_id=task_id, auth=asana_auth, return_raw=False)

AsanaTask(id='1206043185525463', name='Draft project brief', workspace_id='1206043147237389', assignee=AsanaUser(id='1206043147237379', name='Jae Wilson', email=None), assignee_status='inbox', is_completed=False, created_at=datetime.datetime(2023, 11, 27, 22, 34, 14, 216000, tzinfo=tzutc()), completed_on=None, due_on=datetime.datetime(2023, 11, 29, 0, 0), modified_at=datetime.datetime(2023, 11, 29, 7, 11, 25, 487000, tzinfo=tzutc()), memberships=[AsanaMembership(project=AsanaProject(id='1206043185525448', name='asana_sync', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None), section=AsanaSection(id='1206043185525449', name='To do', resource_type='section'))], notes='Do the thing mr. wilson', parent=None, permalink_url='https://app.asana.com/0/1206043185525448/1206043185525463', projects=[AsanaProject(id='1206043185525448', name='asana_sync', workspace_id=None, p

In [ ]:
# | exporti
@patch_to(AsanaTask)
async def get_stories(
    self,
    debug_api: bool = False,
    return_raw: bool = False,
    is_only_comments: bool = False,  # stories include any changes to the task, comments are a subtype of stories
):
    auth = self.auth

    url = f"{auth.base_url}/tasks/{self.id}/stories"

    res = await gd.get_data(
        auth=self.auth,
        method="GET",
        url=url,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    self.stories = [
        AsanaStory._from_json(story_obj, auth=auth)
        for story_obj in res.response["data"]
    ]

    if is_only_comments:
        self.stories = [
            story for story in self.stories if "comment" in story.resource_subtype
        ]

    return self.stories

In [ ]:
show_doc(get_stories)

---

[source](https://github.com/fastai/nbdev/blob/master/nbdev/showdoc.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### show_doc

>      show_doc (sym, renderer=None, name:str|None=None, title_level:int=3)

Show signature and docstring for `sym`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sym |  |  | Symbol to document |
| renderer | NoneType | None | Optional renderer (defaults to markdown) |
| name | str \| None | None | Optionally override displayed name of `sym` |
| title_level | int | 3 | Heading level to use for symbol name |

In [ ]:
asana_pat = json.loads(os.environ["ASANA_PUBLIC"])["ASANA_PAT"]
workspace_id = "1206043185525448"
asana_auth = aa.AsanaAuth(token=asana_pat, workspace_id=workspace_id)

task_id = "1206043185525463"
asana_task = await AsanaTask.get_by_id(
    task_id=task_id, auth=asana_auth, return_raw=False
)

await asana_task.get_stories(return_raw=False, is_only_comments=True, debug_api=False)

[AsanaStory(id='1206043165446912', created_at=datetime.datetime(2023, 11, 27, 22, 49, 29, 310000, tzinfo=tzutc()), created_by=AsanaUser(id='1206043147237379', name='Jae Wilson', email=None), text='we need to get this done fast!', type='comment', resource_subtype='comment_added')]

In [ ]:
# | exporti
def handle_render_user(mdFile, key, value):
    mdFile.new_line(f"{key} - {value.name}")


def handle_render_datetime(mdFile, key, value):
    mdFile.new_line(f"{key} - {value.date()}")


render_factory_values = {
    "AsanaUser": handle_render_user,
    "datetime": handle_render_datetime,
}


def handle_render_membership(mdFile, key, value):
    mdFile.new_header(level=1, title=key)
    [mdFile.new_line(asana_class.to_text()) for asana_class in value]


def handle_render_stories(mdFile, key, value):
    mdFile.new_header(level=1, title=key)
    [
        mdFile.new_line(asana_class.to_text())
        for asana_class in value
        if "comment" in asana_class.resource_subtype
    ]


def handle_render_default(mdFile, key, value):
    mdFile.new_line(f"{key} - {value}")


render_factory_keys = {
    "memberships": handle_render_membership,
    "stories": handle_render_stories,
    "default": handle_render_default,
}


def render_field(key: str, value: str, mdFile):
    if key in ["name", "projects"] or not value:
        return

    render_fn = (
        render_factory_values.get(value.__class__.__name__)
        or render_factory_keys.get(key)
        or render_factory_keys.get("default")
    )

    render_fn(mdFile=mdFile, key=key, value=value)


@patch_to(AsanaTask)
def to_md(self, output_folder="markdown", output_file=None):
    ut.upsert_folder(output_folder)

    mdFile = MdUtils(
        file_name=f"{output_folder}/{output_file or self.id}",
        title=output_file or self.name,
    )

    [
        render_field(key, getattr(self, key), mdFile)
        for key in self.__dict__.keys()
        if key not in ["auth", "workspace_id"]
    ]
    mdFile.create_md_file()

    return f"done exporting {mdFile.file_name}"
    # mdFile.new_table(columns=3, rows=6, text=list_of_strings, text_align='center')

In [ ]:
show_doc(AsanaTask.to_md)

---

[source](https://github.com/jaewilson07/domolibrary_extensions/blob/main/domolibrary_extensions/asana/task.py#L275){target="_blank" style="float:right; font-size:smaller"}

### AsanaTask.to_md

>      AsanaTask.to_md (output_folder='markdown', output_file=None)

In [ ]:
await asana_task.get_stories()

asana_task.to_md(output_folder="../TEST/asana/task/", output_file="sample_task.md")

'done exporting ../TEST/asana/task//sample_task.md'

# Reverse Patch Projects

In [ ]:
# | exporti
@patch_to(ap.AsanaProject)
async def get_tasks(
    self: ap.AsanaProject, debug_api: bool = False, return_raw: bool = False
):
    import domolibrary_extensions.asana.task as at

    auth = self.auth
    params = {"project": self.id}

    url = f"{auth.base_url}/tasks"

    res = await gd.get_data(
        auth=auth, method="GET", url=url, debug_api=debug_api, params=params
    )

    if return_raw:
        return res

    return [
        AsanaTask._from_json(task_obj, auth=auth) for task_obj in res.response["data"]
    ]

In [ ]:
asana_pat = json.loads(os.environ["ASANA_PUBLIC"])["ASANA_PAT"]
workspace_id = "1206043185525448"
project_id = "1206043185525448"

asana_auth = aa.AsanaAuth(token=asana_pat, workspace_id=workspace_id)

asana_project = await ap.AsanaProject.get_by_id(auth=asana_auth, project_id=project_id)

await asana_project.get_tasks()

[AsanaTask(id='1206043185525463', name='Draft project brief', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], stories=None, tags=[]),
 AsanaTask(id='1206043185525465', name='Schedule kickoff meeting', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], stories=None, tags=[]),
 AsanaTask(id='1206043185525467', name='Share timeline with teammates', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], stories=None, tags=[])]

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)